In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
df = pd.read_csv('movies_with_test.csv')

C:\Users\minht\AppData\Local\Temp\ipykernel_23340\2877273640.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_with_test.csv')


In [3]:
df.shape

(930739, 2)

In [4]:
def create_shingles(text, min_length=1, max_length=None):
  tags = [tag.strip() for tag in text.split(',')]  # Tách các tag riêng biệt

  all_shingles = set()
  for tag in tags:
      words = tag.split()  # Tách tag thành các từ
      if max_length is None:
          max_length = max(len(word) for word in words)
      for word in words:
          for i in range(min_length, min(max_length, len(word)) + 1):
              all_shingles.add(word[:i])
  return all_shingles

df['shingles'] = df['genres'].astype(str).apply(create_shingles)

In [5]:
df['shingles']

0                         {Comed, Co, Come, Com, C, Comedy}
1                                 {Dra, Dram, Drama, D, Dr}
2         {Comed, Co, Adu, Come, Com, Ad, C, Adul, Adult...
3         {Anim, Fam, Animation, Animati, Fa, Animatio, ...
4                                 {Dra, Dram, Drama, D, Dr}
                                ...                        
930734                                         {n, nan, na}
930735                                         {n, nan, na}
930736                                         {n, nan, na}
930737                                         {n, nan, na}
930738                                         {n, nan, na}
Name: shingles, Length: 930739, dtype: object

In [6]:
def create_hash_functions(num_hash_functions, max_val):
    hash_functions = []
    for _ in range(num_hash_functions):
        a = random.randint(1, max_val)
        b = random.randint(0, max_val)
        hash_functions.append(lambda x, a=a, b=b: (a * x + b) % max_val)
    return hash_functions

def minhash_signature(shingles, hash_functions):
    signature = [float('inf')] * len(hash_functions) 
    for shingle in shingles:
        shingle_hash = hash(shingle)
        for i, hash_func in enumerate(hash_functions):
            signature[i] = min(signature[i], hash_func(shingle_hash))
    return signature

all_shingles = set()
for shingles_list in df['shingles']:
    all_shingles.update(shingles_list)
# Create 10 hash functions
hash_functions = create_hash_functions(10, len(all_shingles))

# Apply minhash_signature to each row in the `shingles` column
df['minhash_signature'] = df['shingles'].apply(lambda x: minhash_signature(x, hash_functions))


In [7]:
all_shingles

{'A',
 'Ac',
 'Act',
 'Acti',
 'Actio',
 'Action',
 'Ad',
 'Adu',
 'Adul',
 'Adult',
 'Adv',
 'Adve',
 'Adven',
 'Advent',
 'Adventu',
 'Adventur',
 'Adventure',
 'An',
 'Ani',
 'Anim',
 'Anima',
 'Animat',
 'Animati',
 'Animatio',
 'Animation',
 'B',
 'Bi',
 'Bio',
 'Biog',
 'Biogr',
 'Biogra',
 'Biograp',
 'Biograph',
 'Biography',
 'C',
 'Co',
 'Com',
 'Come',
 'Comed',
 'Comedy',
 'Cr',
 'Cri',
 'Crim',
 'Crime',
 'D',
 'Do',
 'Doc',
 'Docu',
 'Docum',
 'Docume',
 'Documen',
 'Document',
 'Documenta',
 'Documentar',
 'Documentary',
 'Dr',
 'Dra',
 'Dram',
 'Drama',
 'F',
 'Fa',
 'Fam',
 'Fami',
 'Famil',
 'Family',
 'Fan',
 'Fant',
 'Fanta',
 'Fantas',
 'Fantasy',
 'G',
 'Ga',
 'Gam',
 'Game',
 'Game-',
 'Game-S',
 'Game-Sh',
 'Game-Sho',
 'Game-Show',
 'H',
 'Hi',
 'His',
 'Hist',
 'Histo',
 'Histor',
 'History',
 'Ho',
 'Hor',
 'Horr',
 'Horro',
 'Horror',
 'M',
 'Mu',
 'Mus',
 'Musi',
 'Music',
 'Musica',
 'Musical',
 'My',
 'Mys',
 'Myst',
 'Myste',
 'Myster',
 'N',
 'Ne',
 'Ne

In [8]:
class LSH:
    def __init__(self, num_bands, num_rows_per_band):
        self.num_bands = num_bands
        self.num_rows_per_band = num_rows_per_band
        self.buckets = {}  # Lưu trữ các nhóm (bucket)


    def hash_function(self, signature, band_index):
        """
        Hàm băm đơn giản để ánh xạ một phần của chữ ký MinHash vào một nhóm.
        """
        start_row = band_index * self.num_rows_per_band
        end_row = start_row + self.num_rows_per_band
        band_signature = tuple(signature[start_row:end_row])
        return hash(band_signature)

    def insert(self, key, signature):
        """
        Chèn một mục (key) và chữ ký MinHash của nó vào các nhóm LSH.
        """
        for band_index in range(self.num_bands):
            bucket_id = self.hash_function(signature, band_index)
            if bucket_id not in self.buckets:
                self.buckets[bucket_id] = []
            self.buckets[bucket_id].append(key)

    def movie_similar(self, query_signature):
        """
        Truy vấn các mục có khả năng tương tự dựa trên chữ ký MinHash của truy vấn.
        """
        candidates = set()
        for band_index in range(self.num_bands):
            bucket_id = self.hash_function(query_signature, band_index)
            if bucket_id in self.buckets:
                candidates.update(self.buckets[bucket_id])
        return candidates
    def movie_query(self, tag_query):
        """
        Truy vấn các bộ phim có khả năng tương tự dựa trên tag truy vấn.
        """

        # Tạo tập hợp shingles từ tag truy vấn
        query_shingles = create_shingles(tag_query)

        # Chuyển đổi shingles thành MinHash signature (sử dụng hash_functions của LSH)
        query_signature = minhash_signature(query_shingles, hash_functions)    

        # Tìm các bộ phim ứng viên bằng LSH
        candidate_movie_ids = self.movie_similar(query_signature)

        return candidate_movie_ids

In [9]:
def jaccard_similarity(set1, set2):
    """
    Tính toán độ tương đồng Jaccard giữa hai tập hợp
    """
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

In [10]:
lsh = LSH(num_bands=5, num_rows_per_band=2)  # Chia chữ ký MinHash thành 5 bands, mỗi band có 2 hàng

# Chèn các bộ phim vào LSH
for index, row in df.iterrows():
    lsh.insert(row['originalTitle'], row['minhash_signature'])

# Truy vấn phim tương tự
query_movie = 'The Millionaire'
query_signature = df[df['originalTitle'] == query_movie]['minhash_signature'].iloc[0]
candidate_movies = lsh.movie_similar(query_signature)
print(candidate_movies)


{'The Leafblower', 'Episode dated 6 May 2005', 'War on Three Fronts', 'The Pythoness', 'Moving McAllister', 'Décembre', 'Sinfonia Amazônica', 'One Piece: Oounabara ni hirake! Dekkai dekkai chichi no yume!', 'Macao', 'Diahann Carroll: No Strings', 'Alas sobre El Chaco', 'Destiny Love', 'The Millionaire', 'The Target Shoots First', 'Chapter 15', 'The Utilizer', 'The Brothers Gruff', 'The Sword and the Dragon', 'The Mystery of Edward Sims: Part 2', 'Niji no ike no jigoku janguru', 'A Fox in a Fix', 'Tobunda ôzora he', 'Synthetic Love', 'The Ixtafa Affair', 'Episode dated 10 January 2015', 'Super Dave: Daredevil for Hire'}


In [11]:
similarities = []
for candidate_movie in candidate_movies:
    candidate_signature = df[df['originalTitle'] == candidate_movie]['minhash_signature'].iloc[0]
    similarity = jaccard_similarity(set(query_signature), set(candidate_signature)) / len(hash_functions)
    similarities.append((candidate_movie, similarity))

threshold = 0.1
filtered_movies = [movie for movie, sim in similarities if sim >= threshold]
for movie, similarity in similarities:
    if movie != query_movie:  # Loại trừ chính bộ phim truy vấn
        print(f"Movie: {movie}, Jaccard Similarity: {similarity}")

Movie: The Leafblower, Jaccard Similarity: 0.05454545454545454
Movie: Episode dated 6 May 2005, Jaccard Similarity: 0.05454545454545454
Movie: War on Three Fronts, Jaccard Similarity: 0.045454545454545456
Movie: The Pythoness, Jaccard Similarity: 0.045454545454545456
Movie: Moving McAllister, Jaccard Similarity: 0.045454545454545456
Movie: Décembre, Jaccard Similarity: 0.013333333333333332
Movie: Sinfonia Amazônica, Jaccard Similarity: 0.06
Movie: One Piece: Oounabara ni hirake! Dekkai dekkai chichi no yume!, Jaccard Similarity: 0.03636363636363636
Movie: Macao, Jaccard Similarity: 0.03636363636363636
Movie: Diahann Carroll: No Strings, Jaccard Similarity: 0.03636363636363636
Movie: Alas sobre El Chaco, Jaccard Similarity: 0.03333333333333333
Movie: Destiny Love, Jaccard Similarity: 0.03636363636363636
Movie: The Target Shoots First, Jaccard Similarity: 0.06
Movie: Chapter 15, Jaccard Similarity: 0.05
Movie: The Utilizer, Jaccard Similarity: 0.038461538461538464
Movie: The Brothers Gru

In [12]:
tag_query = "Ad"
query_result = lsh.movie_query(tag_query)
print(query_result)

{'Tit Teazer Files 1', 'Swedish Erotica 9', 'White Trash Whore 26', 'Meat My Ass 3', 'Botas Texanas y balas salvajes', 'Der Hausmeister', 'World Sex Tour 22: Jamaica'}
